Toronto Neighborhood Segmentation

Toronto Neighborhood Segmentation Question 1

In [1]:
#For this questions, all boroughs with a value of Not assigned were removed from the data
#Additionally, for Neighborhoods with a value of Not assigned, the borough name was assigned as the Neighbohood name

import pandas as pd

df = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")[0]

df=pd.DataFrame(df)
df = df[df.Borough != 'Not assigned'] #creates new dataframe with Boroughs removed that had Not assigned as a value

n=0
for row in df.Neighborhood: #iterates through Neighborhoods and replaces Not Assigned with borough.
    if row=="Not assigned":
        df.iloc[n,2]=df.iloc[n,1]
    n=n+1
    
df.reset_index(drop=True,inplace=True) #resets the index so it is sequential.

df.head()



,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [2]:
df.shape #prints the shape

(210, 3)

Toronto Neighborhood Segmentation Question 2

In [3]:
import pandas as pd
#The Geocode link was not working so I used the csv option.

csv_path = "https://cocl.us/Geospatial_data"
df_latlong=pd.read_csv(csv_path)  #creates a dataframe of the csv coordinates for each area.

df['Latitude']=1 #adds the column Latitude to the Toronto dataframe
df['Longitude']=1 #adds the column Longitude to the Toronto dataframe

n=0
for row in df.Postcode:
    geo_line=(df_latlong[df_latlong['Postal Code'] == row]) #looks up the lat long for the matching row
    df.iloc[n,3]=(geo_line.iloc[0,1]) #latitude
    df.iloc[n,4]=(geo_line.iloc[0,2]) #longitude  
    n=n+1
    
df.head()  #prints the dataframe

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Heights,43.718518,-79.464763
4,M6A,North York,Lawrence Manor,43.718518,-79.464763


Toronto Neighborhood Segmentation Question 3

In [4]:
!pip install folium
!conda install -c conda-forge geopy -yes
!conda install -c conda-forge folium=0.5.0 --yes

usage: conda install [-h] [--revision REVISION] [-y] [--dry-run] [-f]
                     [--file FILE] [--no-deps] [--only-deps] [-m] [-C]
                     [--use-local] [--offline] [--no-pin] [-c CHANNEL]
                     [--override-channels] [-n ENVIRONMENT | -p PATH] [-q]
                     [--copy] [-k] [--update-dependencies]
                     [--no-update-dependencies] [--channel-priority]
                     [--no-channel-priority] [--clobber] [--show-channel-urls]
                     [--no-show-channel-urls] [--download-only] [--json]
                     [--debug] [--verbose]
                     [package_spec [package_spec ...]]
conda install: error: argument -y/--yes: ignored explicit argument 'es'
Solving environment: done

# All requested packages already installed.



In [5]:
!pip install xlrd

In [6]:
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import json
import folium

The code below plots each of the neighborhoods and boroughs on the map.  

In [26]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(width="100%", height="100%", location=[42.853259, -79.329656], zoom_start=9) #tiles= 'Stamen Terrain'


# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
  
map_toronto


In [30]:
CLIENT_ID = 'QCLOQTK3WRLR4IDGCOF2SWYP2JD2DKVGNQUB1WLBSWWMH4GY' # your Foursquare ID
CLIENT_SECRET = 'IEGKT2SRAZMJWGS2OTOM2U4OESQLHGFTDHFNAR2PAF4NHY0M' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

LIMIT = 100 
radius = 500

import json
from pandas.io.json import json_normalize
import requests

Your credentails:
CLIENT_ID: QCLOQTK3WRLR4IDGCOF2SWYP2JD2DKVGNQUB1WLBSWWMH4GY
CLIENT_SECRET:IEGKT2SRAZMJWGS2OTOM2U4OESQLHGFTDHFNAR2PAF4NHY0M


In [31]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [32]:
# type your answer here

toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )



Parkwoods
Victoria Village
Harbourfront
Lawrence Heights
Lawrence Manor
Queen's Park
Queen's Park
Rouge
Malvern
Don Mills North
Woodbine Gardens
Parkview Hill
Ryerson
Garden District
Glencairn
Cloverdale
Islington
Martin Grove
Princess Gardens
West Deane Park
Highland Creek
Rouge Hill
Port Union
Flemingdon Park
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens
Eringate
Markland Wood
Old Burnhamthorpe
Guildwood
Morningside
West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor
Downsview North
Wilson Heights
Thorncliffe Park
Adelaide
King
Richmond
Dovercourt Village
Dufferin
Scarborough Village
Fairview
Henry Farm
Oriole
Northwood Park
York University
East Toronto
Harbourfront East
Toronto Islands
Union Station
Little Portugal
Trinity
East Birchmount Park
Ionview
Kennedy Park
Bayview Village
CFB Toronto
Downsview East
The Danforth West
Riverdale
Design Exchange
Toronto 

In [34]:
print(toronto_venues.shape)
toronto_venues.head()

(4339, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,GTA Restoration,43.753396,-79.333477,Fireworks Store
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Parkwoods,43.753259,-79.329656,TTC stop - 44 Valley Woods,43.755402,-79.333741,Bus Stop
4,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena


In [65]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,100,100,100,100,100,100
Agincourt,5,5,5,5,5,5
Agincourt North,3,3,3,3,3,3
Albion Gardens,9,9,9,9,9,9
Alderwood,8,8,8,8,8,8
Bathurst Manor,19,19,19,19,19,19
Bathurst Quay,16,16,16,16,16,16
Bayview Village,4,4,4,4,4,4
Beaumond Heights,9,9,9,9,9,9


In [36]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 277 uniques categories.


In [66]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [67]:
toronto_onehot.shape

(4339, 277)

In [68]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Adelaide,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.02,...,0.0,0.02,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.01
1,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00
2,Agincourt North,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00
3,Albion Gardens,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00
4,Alderwood,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00


In [69]:
toronto_grouped.shape

(199, 277)

In [70]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
          venue  freq
0   Coffee Shop  0.08
1           Bar  0.04
2          Café  0.04
3    Steakhouse  0.04
4  Burger Joint  0.03


----Agincourt----
                       venue  freq
0             Clothing Store   0.2
1  Latin American Restaurant   0.2
2               Skating Rink   0.2
3                     Lounge   0.2
4             Breakfast Spot   0.2


----Agincourt North----
                      venue  freq
0                      Park  0.33
1                Playground  0.33
2               Coffee Shop  0.33
3               Yoga Studio  0.00
4  Mediterranean Restaurant  0.00


----Albion Gardens----
                  venue  freq
0         Grocery Store  0.22
1          Liquor Store  0.11
2            Beer Store  0.11
3  Fast Food Restaurant  0.11
4              Pharmacy  0.11


----Alderwood----
          venue  freq
0   Pizza Place  0.25
1      Pharmacy  0.12
2           Gym  0.12
3  Skating Rink  0.12
4           Pub  0.12


----Bathurst Manor----
         

In [71]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [72]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Bar,Steakhouse,Café,Sushi Restaurant,Thai Restaurant,Burger Joint,Asian Restaurant,Bakery,Hotel
1,Agincourt,Latin American Restaurant,Breakfast Spot,Skating Rink,Lounge,Clothing Store,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant
2,Agincourt North,Park,Coffee Shop,Playground,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
3,Albion Gardens,Grocery Store,Fried Chicken Joint,Pharmacy,Sandwich Place,Pizza Place,Beer Store,Fast Food Restaurant,Liquor Store,Electronics Store,Eastern European Restaurant
4,Alderwood,Pizza Place,Coffee Shop,Gym,Skating Rink,Pub,Pharmacy,Sandwich Place,Dog Run,Dessert Shop,Dim Sum Restaurant


In [96]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 1, 2, 2, 2, 2, 2, 2, 2], dtype=int32)

In [98]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_) 

toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0.0,Park,Bus Stop,Fireworks Store,Food & Drink Shop,Women's Store,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Grocery Store,Portuguese Restaurant,Coffee Shop,Hockey Arena,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0.0,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Café,Mexican Restaurant,Brewery,Shoe Store,Restaurant
3,M6A,North York,Lawrence Heights,43.718518,-79.464763,0.0,Clothing Store,Gift Shop,Boutique,Furniture / Home Store,Coffee Shop,Women's Store,Miscellaneous Shop,Vietnamese Restaurant,Accessories Store,Doner Restaurant
4,M6A,North York,Lawrence Manor,43.718518,-79.464763,0.0,Clothing Store,Gift Shop,Boutique,Furniture / Home Store,Coffee Shop,Women's Store,Miscellaneous Shop,Vietnamese Restaurant,Accessories Store,Doner Restaurant


In [100]:
# create map
map_clusters = folium.Map(location=[43.653259, -79.329656], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow,#[cluster-1],
        fill=True,
        fill_color=rainbow,#[cluster*100],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [83]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,0.0,Park,Bus Stop,Fireworks Store,Food & Drink Shop,Women's Store,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant
1,North York,0.0,Grocery Store,Portuguese Restaurant,Coffee Shop,Hockey Arena,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
2,Downtown Toronto,0.0,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Café,Mexican Restaurant,Brewery,Shoe Store,Restaurant
3,North York,0.0,Clothing Store,Gift Shop,Boutique,Furniture / Home Store,Coffee Shop,Women's Store,Miscellaneous Shop,Vietnamese Restaurant,Accessories Store,Doner Restaurant
4,North York,0.0,Clothing Store,Gift Shop,Boutique,Furniture / Home Store,Coffee Shop,Women's Store,Miscellaneous Shop,Vietnamese Restaurant,Accessories Store,Doner Restaurant
5,Downtown Toronto,0.0,Coffee Shop,Gym,Park,Yoga Studio,Diner,Beer Bar,Fast Food Restaurant,Italian Restaurant,Smoothie Shop,Sandwich Place
6,Queen's Park,0.0,Coffee Shop,Gym,Park,Yoga Studio,Diner,Beer Bar,Fast Food Restaurant,Italian Restaurant,Smoothie Shop,Sandwich Place
9,North York,0.0,Gym / Fitness Center,Caribbean Restaurant,Japanese Restaurant,Café,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
10,East York,0.0,Fast Food Restaurant,Pizza Place,Gastropub,Pet Store,Pharmacy,Café,Breakfast Spot,Intersection,Bank,Athletics & Sports
11,East York,0.0,Fast Food Restaurant,Pizza Place,Gastropub,Pet Store,Pharmacy,Café,Breakfast Spot,Intersection,Bank,Athletics & Sports


In [84]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Scarborough,1.0,Fast Food Restaurant,Women's Store,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
8,Scarborough,1.0,Fast Food Restaurant,Women's Store,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant


In [85]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
59,East York,2.0,Park,Coffee Shop,Convenience Store,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
69,North York,2.0,Park,Airport,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
70,North York,2.0,Park,Airport,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
115,York,2.0,Park,Convenience Store,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Department Store,Drugstore
119,North York,2.0,Park,Bank,Convenience Store,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
154,Scarborough,2.0,Park,Coffee Shop,Playground,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
155,Scarborough,2.0,Park,Coffee Shop,Playground,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
156,Scarborough,2.0,Park,Coffee Shop,Playground,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
157,Scarborough,2.0,Park,Coffee Shop,Playground,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
182,Downtown Toronto,2.0,Park,Trail,Playground,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant


In [86]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
105,North York,3.0,Baseball Field,Women's Store,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Dim Sum Restaurant
106,North York,3.0,Baseball Field,Women's Store,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Dim Sum Restaurant


In [87]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
97,North York,4.0,Business Service,Baseball Field,Food Truck,Women's Store,Fast Food Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
197,Etobicoke,4.0,Park,Business Service,Baseball Field,Farmers Market,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
198,Etobicoke,4.0,Park,Business Service,Baseball Field,Farmers Market,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
199,Etobicoke,4.0,Park,Business Service,Baseball Field,Farmers Market,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
200,Etobicoke,4.0,Park,Business Service,Baseball Field,Farmers Market,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
201,Etobicoke,4.0,Park,Business Service,Baseball Field,Farmers Market,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
202,Etobicoke,4.0,Park,Business Service,Baseball Field,Farmers Market,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
203,Etobicoke,4.0,Park,Business Service,Baseball Field,Farmers Market,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
204,Etobicoke,4.0,Park,Business Service,Baseball Field,Farmers Market,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
